In [ ]:
import math
from queue import Queue

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
maze = lines.copy()

In [ ]:
start_point = (-1, -1)
end_point = (-1, -1)
for i in range(0, len(maze[0])):
    if maze[0][i] == ".":
        start_point = (0, i)

for i in range(0, len(maze[0])):
    if maze[-1][i] == ".":
        end_point = (len(maze) - 1, i)

print(start_point, end_point)

In [ ]:
dirs = {">": (0, 1), "^": (-1, 0), "v": (1, 0), "<": (0, -1)}


def mv_pos(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1])


def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


def get_field(pos, field):
    return field[pos[0]][pos[1]]


def print_maze(visited_fields, maze):
    for i in range(0, len(maze)):
        for j in range(0, len(maze[0])):
            if (i, j) in visited_fields:
                print("O", end="")
            else:
                print(maze[i][j], end="")
        print()


def get_path(start, target, predecesors):
    path = []
    pred = predecesors[target]
    while start != pred:
        path.append(pred)
        pred = predecesors[pred]
    path.append(start)
    return list(reversed(path))

In [ ]:
# create a graph


def get_neighbours(pos, maze):
    neighbours = []
    for dir in dirs.values():
        neigh_pos = mv_pos(pos, dir)
        if not in_bounds(neigh_pos, maze) or neigh_pos == pos:
            continue

        neigh_field = get_field(neigh_pos, maze)
        if neigh_field != "#":
            if neigh_field not in dirs.keys():
                neighbours.append(mv_pos(pos, dir))
            else:
                if dirs[neigh_field] != (dir[0] * -1, dir[1] * -1):
                    neighbours.append(mv_pos(pos, dir))
    return neighbours


nodes = set()
edges = set()

q = Queue()
q.put(start_point)

while not q.empty():
    n = q.get()
    if n in nodes:
        continue

    nodes.add(n)
    neighours = get_neighbours(n, maze)
    for neigh in neighours:
        if ((neigh, n)) not in edges:
            edges.add((n, neigh))
        q.put(neigh)

In [ ]:
# Bellman-Ford

dists = {n: math.inf for n in nodes}
dists[start_point] = 0

predecesor = {n: None for n in nodes}

node_count = len(nodes)
edge_count = len(edges)

for i in range(0, node_count - 1):
    for e in edges:
        if dists[e[0]] + (-1) < dists[e[1]]:
            dists[e[1]] = dists[e[0]] + (-1)
            predecesor[e[1]] = e[0]

for e in edges:
    if dists[e[0]] + (-1) < dists[e[1]]:
        print("Neg cycle found")
        break

In [ ]:
dists[end_point] * -1

In [ ]:
def get_path(start, target, predecesors):
    path = []
    pred = predecesors[target]
    while start != pred:
        path.append(pred)
        pred = predecesors[pred]
    path.append(start)
    return list(reversed(path))


# path = get_path(start_point, end_point, predecesor)
# print_maze(set(path), maze)

part 2

In [ ]:
# This longest path problem is NP hard and the way edges and vertices are computed right now is a problem for that
# Lets try to significantly reduce the graph size, by only having nodes that branch as nodes and compute the dist between them
# And then good old brute force search

In [ ]:
def get_neighbours(pos, maze):
    neighbours = []
    for dir in dirs.values():
        neigh_pos = mv_pos(pos, dir)
        if not in_bounds(neigh_pos, maze) or neigh_pos == pos:
            continue

        neigh_field = get_field(neigh_pos, maze)
        if neigh_field != "#":
            neighbours.append(mv_pos(pos, dir))

    return neighbours


def is_branching_node(pos):
    n_count = 0
    for dir in dirs.values():
        n_pos = mv_pos(pos, dir)
        if in_bounds(n_pos, maze) and get_field(n_pos, maze) != "#":
            n_count += 1
    return n_count > 2


def find_next_branch_neighbours(pos):
    pos_neighbours = get_neighbours(pos, maze)
    next_branch_nodes = []
    for neigh in pos_neighbours:
        nodes_visited = set([pos])
        path_node = neigh
        while not is_branching_node(path_node):
            nodes_visited.add(path_node)

            neighneigh = get_neighbours(path_node, maze)
            unvisited_neighneigh = [n for n in neighneigh if n not in nodes_visited]
            if len(unvisited_neighneigh) == 0:
                # only happens on edge nodes
                # because edge node is not a branching node it is wrongly counted and needs to be removed
                nodes_visited = list(nodes_visited)[:-1]
                break
            else:
                # this can only be one if its not branching
                path_node = unvisited_neighneigh[0]
        next_branch_nodes.append((path_node, len(nodes_visited)))
    return next_branch_nodes


# find_next_branch_neighbours((2,1))

In [ ]:
nodes = set()
edges = set()

q = Queue()
q.put(start_point)

while not q.empty():
    n = q.get()
    if n in nodes:
        continue

    nodes.add(n)
    neighours = find_next_branch_neighbours(n)
    for neigh in neighours:
        edges.add(((n, neigh[0]), neigh[1]))
        edges.add(((neigh[0], n), neigh[1]))
        q.put(neigh[0])

# In case of multiple edges between two nodes, take the longer one because in a simple path it always will be chosen anyway
edge_dict = {}
edge_cost_dict = {}
for e in edges:
    if e[0][0] not in edge_dict:
        edge_dict[e[0][0]] = set([e[0][1]])
    else:
        edge_dict[e[0][0]].add(e[0][1])
    edge_cost_dict[e[0]] = max(e[1], edge_cost_dict.get(e[0], 0))

# print(sorted(nodes))
# print(edges)
# print(edge_dict)
# print(edge_cost_dict)

In [ ]:
def find_longest_path(start, end, visited):
    if start == end:
        return 0

    longest_path = -1
    graph_neigh = edge_dict[start]
    for gn in graph_neigh:
        if gn in visited:
            continue
        longest_from_neigh = find_longest_path(gn, end, visited.union([gn]))
        # A valid path from gn was found, check if its longer
        if longest_from_neigh >= 0:
            gn_dist = edge_cost_dict[(start, gn)] + longest_from_neigh
            longest_path = max(gn_dist, longest_path)

    return longest_path


find_longest_path(start_point, end_point, set([start_point]))